In [4]:
import findspark
findspark.init()
import os
import time
import datetime
import pyspark.sql.functions as sf
from uuid import *
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql.functions import when
from pyspark.sql.functions import col
from pyspark.sql.types import *
from pyspark.sql.functions import lit
from pyspark import SparkConf, SparkContext
from uuid import * 
from uuid import UUID
import time_uuid 
from pyspark.sql import Row
from pyspark.sql.functions import udf
from pyspark.sql.functions import monotonically_increasing_id
from pyspark.sql.window import Window as W

In [5]:
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

In [6]:
spark = SparkSession.builder.config("spark.jars.packages",'com.datastax.spark:spark-cassandra-connector_2.12:3.1.0').getOrCreate()

In [7]:
df = spark.read.format("org.apache.spark.sql.cassandra").options(table='tracking',keyspace='mydata').load()

In [8]:
df.show(5)
print(df.count())

+--------------------+----+----------+-----------+---+------------+-----+--------------------+---------------+--------------------+---+--------+----+------+----+------------+----+---------+--------------------+----+--------------------+-------------------+------------+-----------+----------+----------+--------+---+--------+
|         create_time| bid|        bn|campaign_id| cd|custom_track|   de|                  dl|             dt|                  ed| ev|group_id|  id|job_id|  md|publisher_id|  rl|       sr|                  ts|  tz|                  ua|                uid|utm_campaign|utm_content|utm_medium|utm_source|utm_term|  v|      vp|
+--------------------+----+----------+-----------+---+------------+-----+--------------------+---------------+--------------------+---+--------+----+------+----+------------+----+---------+--------------------+----+--------------------+-------------------+------------+-----------+----------+----------+--------+---+--------+
|8d4bc290-0bfc-11e...|

In [9]:
def process_timeuuid(df):
    spark_time = df.select('create_time').collect()
    normal_time = []
    for i in range(len(spark_time)):
        a = time_uuid.TimeUUID(bytes = UUID(spark_time[i][0]).bytes).get_datetime().strftime('%Y-%m-%d %H:%M:%S')
        normal_time.append(a)
    spark_timeuuid = []
    for i in range(len(spark_time)):
        spark_timeuuid.append(spark_time[i][0])
    time_data = spark.createDataFrame(zip(spark_timeuuid,normal_time),['create_time','ts'])
    result = df.join(time_data,['create_time'],'inner').drop(df.ts)
    result = result.select('create_time','ts','bid','job_id','campaign_id','custom_track','group_id','publisher_id')
    return result

In [10]:
process_df=process_timeuuid(df)

In [11]:
process_df.show()

+--------------------+-------------------+----+------+-----------+------------+--------+------------+
|         create_time|                 ts| bid|job_id|campaign_id|custom_track|group_id|publisher_id|
+--------------------+-------------------+----+------+-----------+------------+--------+------------+
|3f796600-0bff-11e...|2022-07-25 09:50:42|   0|  1533|        222|        null|    null|           1|
|b9c95960-0bff-11e...|2022-07-25 09:54:07|   0|  1531|        222|        null|    null|           1|
|b70e2aa0-0986-11e...|2022-07-22 06:22:51|null|  null|       null|        null|    null|        null|
|cb3ade10-018e-11e...|2022-07-12 03:00:32|null|  null|       null|       click|    null|        null|
|e1a01b60-0847-11e...|2022-07-20 16:20:33|null|  null|       null|       click|    null|        null|
|f7963ac0-0d61-11e...|2022-07-27 04:09:53|   0|  1103|        151|       click|      28|           1|
|0c7babc0-fda7-11e...|2022-07-07 03:44:05|   0|   187|         48|       click|   

In [12]:
process_df.cache()

DataFrame[create_time: string, ts: string, bid: int, job_id: int, campaign_id: int, custom_track: string, group_id: int, publisher_id: int]

In [15]:
click_data = process_df.filter(process_df.custom_track == 'click')

In [16]:
click_data.show()

+--------------------+-------------------+----+------+-----------+------------+--------+------------+
|         create_time|                 ts| bid|job_id|campaign_id|custom_track|group_id|publisher_id|
+--------------------+-------------------+----+------+-----------+------------+--------+------------+
|4e51fdb0-089c-11e...|2022-07-21 02:24:54|null|  null|       null|       click|    null|        null|
|5a374880-0b38-11e...|2022-07-24 10:06:57|   1|   188|         48|       click|      34|           1|
|6140b730-0668-11e...|2022-07-18 07:08:09|null|  null|       null|       click|    null|        null|
|95b48750-0193-11e...|2022-07-12 03:34:50|null|  null|       null|       click|    null|        null|
|a49e3a10-0975-11e...|2022-07-22 04:20:39|null|  null|       null|       click|    null|        null|
|bbab1a50-018e-11e...|2022-07-12 03:00:06|null|  null|       null|       click|    null|        null|
|bc3af1b0-0b67-11e...|2022-07-24 15:46:08|null|  null|       null|       click|   

In [23]:
click_data = click_data.na.fill({'bid':0})
click_data = click_data.na.fill({'job_id':0})
click_data = click_data.na.fill({'campaign_id':0})
click_data = click_data.na.fill({'group_id':0})
click_data = click_data.na.fill({'publisher_id':0})

ConnectionRefusedError: [WinError 10061] No connection could be made because the target machine actively refused it